# Aggergate Sql dumps into MongoDB
**Contributors:** Victor Lin

**Achievement:** Migrate osu_user_stats, osu_scores_high, osu_beatmaps tables from several random data dumps into mongodb. Prioritize migrating more recent dumps to avoid duplicates. Created indecies on osu_scores_high

In [1]:
import sys
sys.path.append('../..')
from exploration.config import sql_inst, mongo_inst
from mlpp.data_collection.sql_migration import SqlDumpMigrator

## Declare SQL dumps to migrate
**Note:** Order does matter in the declaration. More recent dumps should be declared first in order to prioritize more recent user information. Duplicate users found in older dumps will be skipped.

In [3]:
# Order of insertion matters

SQL_DUMPS = [
    "osu_random_2021_01",
    "osu_random_2020_12",
    "osu_random_2020_11",
    "osu_random_2020_10",
    "osu_random_2020_09",
    "osu_random_2020_08"
]

NEW_DB_NAME = "osu_random_db"

migrator = SqlDumpMigrator(sql_inst, mongo_inst)

**TODO** : Code to create a compound index on (user_id, beatmap_id, and enabled_mods) would ensure score uniqueness. However, there are currently duplicates scores in some dumps that violates the uniqueness. Commented until further clarification from osu! devs.

In [4]:
# mongo_db['osu_scores_high'].create_index(
#     [('user_id', pymongo.ASCENDING), ('beatmap_id', pymongo.ASCENDING) , ('enabled_mods', pymongo.ASCENDING)],
#     unique=True
# )

## Migrating Tables: Users, Scores, Beatmaps

In [6]:
migrator.migrate_users_and_scores(SQL_DUMPS, NEW_DB_NAME)

In [8]:
migrator.migrate_beatmaps(SQL_DUMPS, NEW_DB_NAME)

## Creating osu_scores_high indecies

In [ ]:
osu_random_db = mongo_inst[NEW_DB_NAME]

print("Creating index on 'user_id'")
osu_random_db['osu_scores_high'].create_index('user_id')

print("Creating index on 'beatmap_id'")
osu_random_db['osu_scores_high'].create_index('beatmap_id')